In [1]:
from __future__ import print_function
import tensorflow as tf
from itertools import izip
from __future__ import print_function
import tensorflow as tf
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from itertools import izip
import nltk
from sklearn.manifold import TSNE
import csv
from nltk.tokenize import TweetTokenizer
import collections
import re
import preprocess_data
import pickle
import numpy as np

%matplotlib inline

In [2]:
with open('../encoder/dictionary-id', 'rb') as load_dict:
    words_dict = pickle.load(load_dict)

print(len(words_dict))

21748


In [3]:
from collections import defaultdict
embedding_dict = defaultdict(list)
embedding = open('../embedding-results/sswe-u.txt')

for line in embedding:
    elements = line.split()
    for i in range(1, len(elements)):
        embedding_dict[elements[0]].append(float(elements[i]))
embedding.close()

embedding_encoder = defaultdict(list)
for k, v in words_dict.iteritems():
    if k in embedding_dict:
        embedding_encoder[v] = embedding_dict[k]

In [4]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_integer("vocabulary_size", 21748, "vocabulary size")
flags.DEFINE_integer("n_sentiment_class", 5, "number sentiment class")
flags.DEFINE_float("starter_learning_rate", 0.001, "starter learning rate")
flags.DEFINE_integer("training_iters", 10000, "number of training iterations")
flags.DEFINE_integer("batch_size", 50, "size of batch")
flags.DEFINE_integer("display_step", 1, "number of steps ==> display")
flags.DEFINE_integer("seq_max_len", 30, "sequence max length")
flags.DEFINE_integer("n_hidden", 200, "number of hidden neurons")
flags.DEFINE_integer("n_hidden_inside", 100, "number of hidden neurons before LSTM")
flags.DEFINE_integer("n_classes", 3, "number of sentiment classes - [0,1,2,3]")
flags.DEFINE_float("dropout", 0.5, "dropout - probability to keep units")
flags.DEFINE_integer("n_add", 4, "number of additional features")
flags.DEFINE_float("amplified_coefficient", 10., "coefficient for adjusting impact of additional features")
flags.DEFINE_boolean("train_mode", True, "training mode is set to True by default")
flags.DEFINE_string("checkpoint_dir", "./", "directory to checkpoint")
flags.DEFINE_float("alpha", 0.001, "regularized term for balancing context and sentiment")
flags.DEFINE_boolean("using_pos", False, "whether using POSTag feature or not")
flags.DEFINE_boolean("using_add_features", False, "whether using added feature or not")
flags.DEFINE_boolean("using_sentiment_masking", True, "whether using sentiment masking")

dimension = FLAGS.vocabulary_size + FLAGS.n_sentiment_class

In [5]:
class SequenceData(object):
    def __init__(self, max_seq_len=200, min_seq_len=1,
                 data_in=None, label_in=None, table=None,
                 add_features=None, pos_flag=None):

        self.data = []
        self.data_2 = []
        self.labels = []
        self.sequence_length = []
        self.add = []

        self.one_hot_out = []
        self.data_target = []
        self.sentiment_masking = []
        self.one_hot_target = []

        normal_class = "5"
        depth1 = len(table)
        depth2 = FLAGS.n_sentiment_class

        if data_in and label_in:

            if pos_flag:
                raise Exception("Please reset pos_flag")

            n_class = FLAGS.n_classes
            with open(data_in) as in_data, open(label_in) as in_label:
                for x, y in izip(in_data, in_label):
                    x = x.strip().split()
                    y = y.strip()
                    length = len(x)
                    assert length >= min_seq_len

                    zero_int = 0
                    one_int = 1
                    zero_mask = 0.01
                    one_mask = 0.99

                    if length > max_seq_len:
                        x = x[:max_seq_len]
                        s = []
                        s_data_2 = []
                        if pos_flag:
                            raise Exception("Please reset pos_flag")
                        else:
                            for i in x:
                                s_ = int(i.split("/")[0])
                                s__ = table[int(i.split("/")[0])]
                                s += [s_]
                                s_data_2 += [s__]

                        self.sequence_length.append(max_seq_len)
                        s_target = []
                        if pos_flag:
                            raise Exception("Please reset pos_flag")

                        else:
                            for i in x[1:]:
                                s_ = int(i.split("/")[0])
                                s_target += [s_]

                        s_one_hot = []
                        s_one_hot += [zero_int for _ in range(max_seq_len - 1)]
                        s_one_hot += [one_int]

                        # using sentiment masking:
                        if FLAGS.using_sentiment_masking:
                            s_sentiment_mask = []
                            for i in x:
                                if i.split("/")[1] == normal_class:
                                    s_sentiment_mask += [zero_mask]
                                else:
                                    s_sentiment_mask += [one_mask]

                        assert len(s_one_hot) == max_seq_len

                    elif length == max_seq_len:
                        s = []
                        s_data_2 = []
                        if pos_flag:
                            raise Exception("Please reset pos_flag")
                        else:
                            for i in x:
                                s_ = int(i.split("/")[0])
                                s__ = table[int(i.split("/")[0])]
                                s += [s_]
                                s_data_2 += [s__]

                        s += [depth1 for _ in range(max_seq_len - length)]
                        s_data_2 += [depth2 for _ in range(max_seq_len - length)]
                        self.sequence_length.append(length)

                        s_target = []
                        if pos_flag:
                            raise Exception("Please reset pos_flag")

                        else:
                            for i in x[1:]:
                                s_ = int(i.split("/")[0])
                                s_target += [s_]

                        s_target += [depth1 for _ in range(max_seq_len - length)]
                        s_one_hot = []
                        s_one_hot += [zero_int for _ in range(length - 1)]
                        s_one_hot += [one_int]

                        # using sentiment masking:
                        if FLAGS.using_sentiment_masking:
                            s_sentiment_mask = []
                            for i in x:
                                if i.split("/")[1] == normal_class:
                                    s_sentiment_mask += [zero_mask]
                                else:
                                    s_sentiment_mask += [one_mask]
                            s_sentiment_mask += [zero_mask for _ in range(max_seq_len - length)]
                    else:
                        s = []
                        s_data_2 = []
                        if pos_flag:
                            raise Exception("Please reset pos_flag")
                        else:
                            for i in x:
                                s_ = int(i.split("/")[0])
                                s__ = table[int(i.split("/")[0])]
                                s += [s_]
                                s_data_2 += [s__]

                        s += [depth1 for _ in range(max_seq_len - length)]
                        s_data_2 += [depth2 for _ in range(max_seq_len - length)]
                        # s += [zero for _ in range(max_seq_len - length)]
                        self.sequence_length.append(length)

                        s_target = []
                        if pos_flag:
                            raise Exception("Please reset pos_flag")
                        else:
                            for i in x[1:]:
                                s_ = int(i.split("/")[0])
                                s_target += [s_]

                        s_target += [depth1 for _ in range(max_seq_len - length)]

                        s_one_hot = []
                        s_one_hot += [zero_int for _ in range(length - 1)]
                        s_one_hot += [one_int]
                        s_one_hot += [zero_int for _ in range(max_seq_len - length)]

                        # using sentiment masking:
                        if FLAGS.using_sentiment_masking:
                            s_sentiment_mask = []
                            for i in x:
                                if i.split("/")[1] == normal_class:
                                    s_sentiment_mask += [zero_mask]
                                else:
                                    s_sentiment_mask += [one_mask]
                            s_sentiment_mask += [zero_mask for _ in range(max_seq_len - length)]

                    s_one_hot_target = []
                    s_one_hot_target += [one_int]
                    s_one_hot_target += [zero_int for _ in range(max_seq_len-1)]

                    assert len(s) == max_seq_len, \
                    "-- Error: length of data is not equal to max_seq_len --"
                    
                    assert len(s_target) == max_seq_len - 1, \
                    "-- Error: length of data_target is invalid --"
                    
                    assert len(s_one_hot) == max_seq_len, \
                    "-- Error: length of one_hot is invalid --"
                    
                    assert len(s_sentiment_mask) == max_seq_len, \
                    "-- Error: length of sentiment-masking is invalid --"
                    
                    self.one_hot_out.append(s_one_hot)
                    self.data_target.append(s_target)
                    self.data.append(s)
                    self.data_2.append(s_data_2)
                    self.sentiment_masking.append(s_sentiment_mask)
                    self.one_hot_target.append(s_one_hot_target)

                    s_label = [0] * n_class
                    s_label[int(y) - 1] = 1
                    self.labels.append(s_label)
            if FLAGS.using_add_features:
                # parsing additional features to sequence data
                with open(add_features) as features_add:
                    for line in features_add:
                        line = line.strip().split()
                        add_feat = [FLAGS.amplified_coefficient * float(i) for i in line]
                        self.add.append(add_feat)
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id + batch_size,
                                                  len(self.data))])
        batch_data_2 = (self.data_2[self.batch_id:min(self.batch_id + batch_size,
                                                      len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id + batch_size,
                                                      len(self.data))])
        batch_sequence_length = (self.sequence_length[self.batch_id:min(self.batch_id + batch_size,
                                                                        len(self.data))])

        batch_one_hot = (self.one_hot_out[self.batch_id:min(self.batch_id + batch_size,
                                                            len(self.data))])

        batch_target = (self.data_target[self.batch_id:min(self.batch_id + batch_size,
                                                           len(self.data))])

        batch_one_hot_target = (self.one_hot_target[self.batch_id:min(self.batch_id + batch_size,
                                                                      len(self.data))])

        if FLAGS.using_add_features:
            batch_add = (self.add[self.batch_id:min(self.batch_id + batch_size,
                                                    len(self.data))])
            self.batch_id = min(self.batch_id + batch_size, len(self.data))
            return batch_data, batch_data_2, batch_labels, batch_sequence_length, batch_add, batch_one_hot, batch_target, batch_one_hot_target

        if FLAGS.using_sentiment_masking:
            batch_sentiment_mask = (self.sentiment_masking[self.batch_id:min(self.batch_id + batch_size,
                                                                             len(self.data))])
            self.batch_id = min(self.batch_id + batch_size, len(self.data))
            return batch_data, batch_data_2, batch_labels, batch_sequence_length, batch_one_hot, batch_target, batch_sentiment_mask, batch_one_hot_target

        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_data_2, batch_labels, batch_sequence_length, batch_one_hot, batch_target, batch_one_hot_target
    

In [6]:
def sparse_encoding(x_main, indices1, indices2):
    depth1 = FLAGS.vocabulary_size
    depth2 = FLAGS.n_sentiment_class
    one_hot_main = tf.cast(indices1, tf.float32)
    one_hot = tf.one_hot(indices=x_main, depth=depth1, on_value=1.0, off_value=0.0, axis=-1)
    two_hot = tf.one_hot(indices=indices2, depth=depth2, on_value=1.0, off_value=0.0, axis=-1)
    merge_main = tf.concat(2, [one_hot_main, two_hot])
    merge = tf.concat(2, [one_hot, two_hot])
   
    # TODO here
    return merge_main, merge


def masking(indices, depth):
    temp = tf.transpose(indices)
    return tf.transpose([temp, ] * depth)


def dynamic_rnn(x_main, x, xx, seqlen,
                weights, biases, dropout, addition,
                one_hot, one_hot_target, sentiment_mask, switch):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unroll data
    
    x, x_tmp = sparse_encoding(x_main, x, xx)
    x = tf.cast(x, tf.float32)

    # one-hot target
    batch_size = tf.shape(x)[0]
    
    x_target = tf.dynamic_partition(x_tmp, one_hot_target, 2)[0]
    
    x_target = tf.reshape(x_target, [batch_size, -1])

    one_hot = masking(one_hot, depth=dimension)
    sentiment_mask = masking(sentiment_mask, depth=FLAGS.n_hidden)
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, 55])
    # passing data through hidden layer with RELU activation
    x = tf.add(tf.matmul(x, weights['inside']), biases['inside'])
    # x = tf.sigmoid(x)
    x = tf.nn.relu(x)
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, FLAGS.seq_max_len, x)
    # Define a lstm cell with tensorflow
    # lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(FLAGS.n_hidden)

    # lstm_cell = tf.nn.rnn_cell.GRUCell(FLAGS.n_hidden)
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(FLAGS.n_hidden)
    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.nn.rnn(lstm_cell, x, dtype=tf.float32, sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.pack(outputs)
    # compute the reconstruction
    # issues: output is slided by 1 element
    # need to share the common LSTM parameters, then using switch may not be appropriate
    # thus, we need to consider using sharing parameters features from Tensorflow
    if switch:
        batch_size = tf.shape(outputs)[1]
        # reshaping to (n_step*batch_size, n_hidden)
        reconstruction = tf.reshape(outputs, [-1, FLAGS.n_hidden])
        reconstruction = tf.add(tf.matmul(reconstruction,
                                          weights['reconstruct']),
                                biases['reconstruct'])
        reconstruction = tf.nn.relu(reconstruction)
        # reconstruction = tf.sigmoid(reconstruction)
        reconstruction = tf.split(0, FLAGS.seq_max_len, reconstruction)
        # change back dimension to [batch_size, n_step, n_input]
        reconstruction = tf.pack(reconstruction)
        reconstruction = tf.transpose(reconstruction, [1, 0, 2])
        # one_hot = tf.to_int32(one_hot)
        reconstruction = tf.dynamic_partition(reconstruction, one_hot, 2)[0]
        # reconstruction = tf.reshape(reconstruction, [-1, FLAGS.seq_max_len-1, dimension])
        # indices = seq_len-1
        reconstruction = tf.reshape(reconstruction, [batch_size, -1])
        # partition_table = tf.one_hot_table(indices, )
        # batch_size = tf.shape(reconstruction)[0]

        reconstruction = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(reconstruction,
                                                                                x_target))
        
        return reconstruction
    else:
        outputs = tf.transpose(outputs, [1, 0, 2])
        

        outputs *= sentiment_mask
        # used = tf.sign(tf.reduce_max(sentiment_mask, reduction_indices=2))
        # length = tf.reduce_sum(used, reduction_indices=1, keep_dims=True)
        # length = tf.cast(length, tf.float32)
        outputs = tf.reduce_sum(outputs, reduction_indices=1)
        # outputs /= length
        if FLAGS.using_add_features:
            outputs = tf.concat(1, [outputs, addition])

        outputs = tf.nn.dropout(outputs, dropout)
        return tf.matmul(outputs, weights['out']) + biases['out']
        '''
        outputs = tf.transpose(outputs, [1, 0, 2])

        outputs *= sentiment_mask

        used = tf.sign(tf.reduce_max(sentiment_mask, reduction_indices=2))

        length = tf.reduce_sum(used, reduction_indices=1, keep_dims=True)

        # length = tf.cast(length, tf.float32)

        outputs = tf.reduce_sum(outputs, reduction_indices=1)

        outputs /= length

        if FLAGS.using_add_features:
            outputs = tf.concat(1, [outputs, addition])

        outputs = tf.nn.dropout(outputs, dropout)
        return tf.matmul(outputs, weights['out']) + biases['out'], length
        '''

In [7]:
def one_hot_table(source):
    dictionary = {}
    count = 1
    with open(source, "r") as infile:
        for line in infile:
            line = line.strip()
            if line:
                dictionary[count] = int(line.split()[1])
                count += 1
    return dictionary


def table_loading():
    source_path = "../encoder/"

    Table = one_hot_table(source_path + "dictionary-list")

    print("===================")
    print("Dynamic RNN for SA")
    print("===================")
    print("Loading lookup table")
    # print(Table)

    # tf Graph input
    print("check input dimension")

    input_size = len(Table) + FLAGS.n_sentiment_class
    print(input_size)
    # assert input_size == dimension, "--check vocabulary length--"

    if FLAGS.using_pos:
        raise Exception("Please reset pos_flag")
    return input_size, Table

In [8]:
def model_training(dimension, lookup_table):
    # clear the default graph stack and reset the global default graph
    print("Reset the global default graph.. ")
    tf.reset_default_graph()
    # Passing data
    print("Passing data")
    source_path = "../encoder/"

    # load training data without additional features
    trainset = SequenceData(max_seq_len=FLAGS.seq_max_len,
                            data_in=source_path + "train-data",
                            label_in=source_path + "train-label",
                            table=lookup_table,
                            add_features=None,
                            pos_flag=FLAGS.using_pos)

    # load test data without additional features
    testset = SequenceData(max_seq_len=FLAGS.seq_max_len,
                           data_in=source_path + "test-data",
                           label_in=source_path + "test-label",
                           table=lookup_table,
                           add_features=None,
                           pos_flag=FLAGS.using_pos)

    print("Check the data size")
    print("Training set:")
    print(len(trainset.data), len(trainset.data[0]))

    print("Test set:")
    print(len(testset.data), len(testset.data[0]))

    test_length = len(testset.data)
    print("Start training ..")

    # Placeholder for data
    x_main = tf.placeholder(tf.int32, [None, FLAGS.seq_max_len])
    x = tf.placeholder(tf.int32, [None, FLAGS.seq_max_len, 50])
    xx = tf.placeholder(tf.int32, [None, FLAGS.seq_max_len])

    # Placeholder for label
    y = tf.placeholder("float", [None, FLAGS.n_classes])

    # A placeholder for indicating each sequence length
    seq_len = tf.placeholder(tf.int32, [None])

    # Placeholder for one-hot partition matrix
    one_hot = tf.placeholder(tf.int32, [None, FLAGS.seq_max_len])

    # Placeholder for one-hot target partition matrix
    one_hot_target = tf.placeholder(tf.int32, [None, FLAGS.seq_max_len])

    # Place holder for sentiment masking
    sentiment_mask = tf.placeholder(tf.float32, [None, FLAGS.seq_max_len])

    # Placeholder for data target
    sent_target = tf.placeholder("float", [None, FLAGS.seq_max_len - 1])
    sentence_target = masking(sent_target, depth=dimension)

    # Placeholder for dropout
    keep_prob = tf.placeholder(tf.float32)

    # Placeholder for added data
    # addition = tf.placeholder("float", [None, FLAGS.n_add])

    # Define weights
    with tf.name_scope("weights"):
        weights = {
            'inside': tf.Variable(tf.random_normal([55, FLAGS.n_hidden_inside])),

            'reconstruct': tf.Variable(tf.random_normal([FLAGS.n_hidden, dimension])),

            'out': tf.Variable(tf.random_normal([FLAGS.n_hidden, FLAGS.n_classes]))
        }
    with tf.name_scope("biases"):
        biases = {
            'inside': tf.Variable(tf.random_normal([FLAGS.n_hidden_inside])),

            'reconstruct': tf.Variable(tf.random_normal([dimension])),

            'out': tf.Variable(tf.random_normal([FLAGS.n_classes]))
        }

    # Variables scope for sharing parameters
    with tf.variable_scope("dynamic_rnn") as scope:
        sentiment = dynamic_rnn(x_main, x, xx, seq_len, weights,
                                biases, keep_prob, addition=None, one_hot=one_hot, 
                                one_hot_target=one_hot_target, sentiment_mask=sentiment_mask,
                                switch=False)
        
        
        scope.reuse_variables()
        reconstruction = dynamic_rnn(x_main, x, xx, seq_len, weights,
                                     biases, keep_prob, addition=None, one_hot=one_hot,
                                     one_hot_target=one_hot_target, sentiment_mask=sentiment_mask, 
                                     switch=True)

    # Define loss and optimizer
    with tf.name_scope("cost_sentiment"):
        cost_sent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(sentiment, y))

    with tf.name_scope("cost_reconstruction"):
        cost_recon = reconstruction

    with tf.name_scope("cost"):
        # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
        cost = FLAGS.alpha * cost_recon + (1 - FLAGS.alpha) * cost_sent


    with tf.name_scope("learning_rate"):
        global_step = tf.Variable(0, trainable=False)
        learning_rate = tf.train.exponential_decay(0.001, global_step, 100, 0.65, staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost,
                                                                          global_step=global_step)

    # Evaluate model
    with tf.name_scope("accuracy"):

        with tf.name_scope("correct_prediction"):
            correct_pred = tf.equal(tf.argmax(sentiment, 1), tf.argmax(y, 1))
            correct_prediction = tf.reduce_sum(tf.cast(correct_pred, tf.float32))
        with tf.name_scope("accuracy"):
            accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

   
    sess = tf.InteractiveSession()


    # Initializing the variables
    init = tf.initialize_all_variables()

    # Using defaults to saving all variables
    saver = tf.train.Saver()

    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)
        saver.restore(sess, "se-v001-test.ckpt")
        if FLAGS.train_mode:
            step = 1
            # Keep training until reach max iterations
            while step * FLAGS.batch_size < 10000:
                if FLAGS.using_add_features:
                    batch_x, batch_xx, batch_y, batch_sequence_length, \
                    batch_add, batch_one_hot, batch_target,\
                    batch_sentiment_mask, batch_one_hot_target = trainset.next(FLAGS.batch_size)

                else:
                    batch_x, batch_xx, batch_y, batch_sequence_length, \
                    batch_one_hot, batch_target, \
                    batch_sentiment_mask, batch_one_hot_target = trainset.next(FLAGS.batch_size)
                

                tmp = np.zeros(shape=(len(batch_x), 30, 50))
                for i in range(len(batch_x)):
                    for j in range(len(batch_x[i])):
                        tmp[i][j] = embedding_encoder[float(batch_x[i][j])]
                # Run optimization (back-prop)
                sess.run(optimizer,
                         feed_dict={x_main: batch_x,
                                    x: tmp,
                                    xx: batch_xx,
                                    y: batch_y,
                                    seq_len: batch_sequence_length,
                                    keep_prob: FLAGS.dropout,
                                    one_hot: batch_one_hot,
                                    sent_target: batch_target,
                                    sentiment_mask: batch_sentiment_mask,
                                    one_hot_target: batch_one_hot_target})
                if step % FLAGS.display_step == 0:
                    # Calculate batch accuracy
                    acc = sess.run(accuracy,
                                   feed_dict={x_main: batch_x,
                                              x: tmp, xx: batch_xx,
                                              y: batch_y,
                                              seq_len: batch_sequence_length,
                                              keep_prob: FLAGS.dropout,
                                              one_hot: batch_one_hot,
                                              sent_target: batch_target,
                                              sentiment_mask: batch_sentiment_mask,
                                              one_hot_target: batch_one_hot_target})


                    # Calculate batch loss
                    loss, loss_sent, loss_recon = sess.run([cost, cost_sent, cost_recon],
                                                           feed_dict={x_main: batch_x,
                                                                  x: tmp,
                                                                  xx: batch_xx,
                                                                  y: batch_y,
                                                                  seq_len: batch_sequence_length,
                                                                  keep_prob: FLAGS.dropout,
                                                                  one_hot: batch_one_hot,
                                                                  sent_target: batch_target,
                                                                  sentiment_mask: batch_sentiment_mask,
                                                                  one_hot_target: batch_one_hot_target})

                    print("Iter " + str(step * FLAGS.batch_size) + ", Loss= " +
                          "{:.4f}".format(loss) + ", Loss Sent= " +
                          "{:.4f}".format(loss_sent) + ", Loss Recon= " +
                          "{:.4f}".format(loss_recon) + ", Accuracy= " +
                          "{:.4f}".format(acc) + ", Learning_rate= " +
                          "{:.6f}".format(learning_rate.eval()))
                    
                        
                step += 1
            print("Optimization Finished!")
            saver.save(sess, "se-v001-test.ckpt")
            # calculate accuracy
            step = 1
            count = 0
            while step * FLAGS.batch_size <= test_length:
                test_x, test_xx, test_y, test_sequence_length, \
                test_one_hot, test_target, test_sentiment_mask, \
                test_one_hot_target = testset.next(FLAGS.batch_size)
                
                tmp = np.zeros(shape=(len(test_x), 30, 50))
                for i in range(len(test_x)):
                    for j in range(len(test_x[i])):
                        tmp[i][j] = embedding_encoder[float(test_x[i][j])]
                        
                        
                correct_pred = sess.run(correct_prediction,
                                        feed_dict={x_main: test_x,
                                                   x: tmp,
                                                   xx: test_xx,
                                                   y: test_y, 
                                                   seq_len: test_sequence_length,
                                                   keep_prob: 1.0,
                                                   one_hot: test_one_hot,
                                                   sent_target: test_target,
                                                   sentiment_mask: test_sentiment_mask,
                                                   one_hot_target: test_one_hot_target})

                print ("Correct prediction: {0} - {1}/ batch_size = {2}".format(step,
                                                                                correct_pred,
                                                                                FLAGS.batch_size))
                count += correct_pred
                step += 1
            print(count)
            print ("Accuracy: {}".format(count/test_length))

In [ ]:
input_size, Table = table_loading()
model_training(dimension=input_size, lookup_table=Table)

Dynamic RNN for SA
Loading lookup table
check input dimension
21753
Reset the global default graph.. 
Passing data
Check the data size
Training set:
6088 30
Test set:
2336 30
Start training ..
Iter 50, Loss= 2.5349, Loss Sent= 2.2195, Loss Recon= 317.5924, Accuracy= 0.5400, Learning_rate= 0.000000
Iter 100, Loss= 3.1690, Loss Sent= 2.8590, Loss Recon= 312.9485, Accuracy= 0.4200, Learning_rate= 0.000000
Iter 150, Loss= 2.8093, Loss Sent= 2.4939, Loss Recon= 317.9645, Accuracy= 0.3600, Learning_rate= 0.000000
Iter 200, Loss= 2.5841, Loss Sent= 2.2538, Loss Recon= 332.5337, Accuracy= 0.4800, Learning_rate= 0.000000
Iter 250, Loss= 2.5702, Loss Sent= 2.2773, Loss Recon= 295.2059, Accuracy= 0.4600, Learning_rate= 0.000000
Iter 300, Loss= 3.2816, Loss Sent= 2.9839, Loss Recon= 300.7059, Accuracy= 0.3600, Learning_rate= 0.000000
Iter 350, Loss= 4.0273, Loss Sent= 3.7004, Loss Recon= 330.6517, Accuracy= 0.4400, Learning_rate= 0.000000
Iter 400, Loss= 2.8348, Loss Sent= 2.5122, Loss Recon= 325.